# Sentiment Analysis MLOps – (Google Colab)

Questo notebook accompagna il progetto di **Sentiment Analysis** sviluppato con un approccio **MLOps**, mostrando l’intero flusso:
dalla preparazione dei dati alla valutazione dei modelli.

- **Modello principale**: Transformer pre-addestrato  
  `cardiffnlp/twitter-roberta-base-sentiment-latest`
- **Baseline di confronto**: modello **FastText**, addestrato specificamente sul dataset del progetto
- **Obiettivo**: dimostrare una pipeline completa, riproducibile e confrontabile per l’analisi del sentiment

📌 **Repository GitHub**  
https://github.com/Nimus74/sentiment-analysis-mlops

ℹ️ Il deploy su **Hugging Face Spaces** è considerato un’estensione opzionale del progetto.

## 1. Setup Ambiente


In [ ]:
!pip install -q "datasets<4.0.0"

# Clone repository
%cd /content
!rm -rf sentiment-analysis-mlops
!git clone https://github.com/Nimus74/sentiment-analysis-mlops.git
%cd sentiment-analysis-mlops

# Install dependencies
!pip install -r requirements.txt -q
!pip install -e . -q

In [ ]:
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.download_dataset import download_dataset
from src.data.preprocessing import preprocess_dataframe
from src.data.validation import validate_dataset_quality
from src.data.split import stratified_split
from src.models.transformer_model import TransformerSentimentModel
from src.models.fasttext_model import FastTextSentimentModel
from src.evaluation.metrics import calculate_metrics, compare_models_metrics


## Confronto tra modelli

Il modello Transformer è utilizzato come approccio principale per l’analisi del sentiment,
mentre FastText è impiegato come baseline supervisionata per confronto.

Questa scelta è motivata dalle migliori prestazioni dei modelli Transformer
su testi brevi e rumorosi come quelli dei social media.

## 2. Download e Preparazione Dataset


In [ ]:
from datasets import load_dataset

# Download dataset (Hugging Face datasets)
# Nota: questo dataset richiede un "config name" (es. 'all', 'italian', 'english', ecc.)
dataset = load_dataset(
    "cardiffnlp/tweet_sentiment_multilingual",
    "all",
    cache_dir="data/raw"
)

df = dataset["train"].to_pandas()

print(f"Dataset scaricato: {len(df)} campioni")
print(f"Colonne: {df.columns.tolist()}")
print("\nDistribuzione classi:")
print(df["label"].value_counts())

In [ ]:
# Preprocessing
df_processed = preprocess_dataframe(
    df,
    text_column="text",
    min_length=3,
    max_length=512
)

print(f"Dopo preprocessing: {len(df_processed)} campioni")


In [ ]:
# Split train/val/test
train_df, val_df, test_df, split_indices = stratified_split(
    df_processed,
    train_size=0.70,
    val_size=0.15,
    test_size=0.15,
    random_seed=42
)

print(f"Train: {len(train_df)} campioni")
print(f"Val: {len(val_df)} campioni")
print(f"Test: {len(test_df)} campioni")


## 3. Training e Valutazione Modelli

### 3.1 Transformer (Pre-addestrato)


In [ ]:
# Carica modello Transformer pre-addestrato
transformer = TransformerSentimentModel(
    model_name="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

print("✅ Transformer caricato")


### 3.2 FastText


In [ ]:
# Prepara formato FastText
from src.data.preprocessing import prepare_fasttext_format

os.makedirs("data/processed", exist_ok=True)
train_file = "data/processed/fasttext_train.txt"
prepare_fasttext_format(
    train_df["text"].tolist(),
    train_df["label"].tolist(),
    train_file
)

# Training FastText
os.makedirs("models/fasttext", exist_ok=True)
fasttext_model = FastTextSentimentModel.train(
    train_file=train_file,
    output_path="models/fasttext/fasttext_model.bin",
    epoch=25,
    lr=0.1
)

print("✅ FastText addestrato")


## 4. Valutazione e Confronto


In [ ]:
# Valutazione su test set
test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()

unique_labels = sorted(test_df["label"].unique())
label_to_num = {label: i for i, label in enumerate(unique_labels)}

# Transformer
transformer_preds = transformer.predict_labels(test_texts)
transformer_metrics = calculate_metrics(
    np.array([label_to_num[l] for l in test_labels]),
    transformer_preds,
    labels=unique_labels
)

print("Transformer Metrics:")
print(f"  Macro-F1: {transformer_metrics['macro_f1']:.4f}")
print(f"  Accuracy: {transformer_metrics['accuracy']:.4f}")


### Nota sulle etichette FastText
FastText produce etichette numeriche (`0/1/2`) in base al dataset di training.
Per questo motivo, nel notebook la conversione viene gestita direttamente prima del calcolo delle metriche,
mantenendo lo stesso spazio di label utilizzato in fase di split e valutazione.

In [ ]:
# FastText - predictions (raw) + conversione numerica robusta
preds = fasttext_model.predict_batch(test_texts)

fasttext_preds = np.array([int(str(p["label"]).replace("__label__", "").strip()) for p in preds])

fasttext_metrics = calculate_metrics(
    np.array([label_to_num[l] for l in test_labels]),
    fasttext_preds,
    labels=unique_labels
)

print("\nFastText Metrics:")
print(f"  Macro-F1: {fasttext_metrics['macro_f1']:.4f}")
print(f"  Accuracy: {fasttext_metrics['accuracy']:.4f}")


In [ ]:
# Confronto modelli
comparison = compare_models_metrics(
    transformer_metrics,
    fasttext_metrics,
    "Transformer",
    "FastText"
)

print("\nConfronto Modelli:")
print(comparison.to_string(index=False))


## 5. Esempi Inferenza


In [ ]:
# Esempi testi
test_examples = [
    "Questo prodotto è fantastico! Lo consiglio a tutti.",
    "Il servizio è stato ok, niente di speciale.",
    "Terribile esperienza, non lo consiglio affatto."
]

print("Esempi Inferenza:\n")
for text in test_examples:
    print(f"Testo: {text}")

    # Transformer
    trans_result = transformer.predict(text)
    print(f"  Transformer: {trans_result['label']} (confidence: {trans_result['score']:.2f})")

    # FastText
    ft_result = fasttext_model.predict(text)[0]
    print(f"  FastText: {ft_result['label']} (confidence: {ft_result['score']:.2f})")


## 6. Visualizzazioni


In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

cm_transformer = confusion_matrix(
    [label_to_num[l] for l in test_labels],
    transformer_preds
)

cm_fasttext = confusion_matrix(
    [label_to_num[l] for l in test_labels],
    fasttext_preds
)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(cm_transformer, annot=True, fmt="d", cmap="Blues", ax=axes[0],
            xticklabels=unique_labels, yticklabels=unique_labels)
axes[0].set_title("Transformer")
axes[0].set_ylabel("True Label")
axes[0].set_xlabel("Predicted Label")

sns.heatmap(cm_fasttext, annot=True, fmt="d", cmap="Blues", ax=axes[1],
            xticklabels=unique_labels, yticklabels=unique_labels)
axes[1].set_title("FastText")
axes[1].set_ylabel("True Label")
axes[1].set_xlabel("Predicted Label")

plt.tight_layout()
plt.show()


## Conclusioni

Il progetto ha portato alla realizzazione di una pipeline completa di **Sentiment Analysis end-to-end**, coprendo tutte le fasi richieste dalla consegna:

- ✅ Pipeline dati completa, riproducibile e modulare
- ✅ Implementazione e confronto tra due approcci distinti:
  - **Transformer** pre-addestrato per sentiment analysis
  - **FastText** addestrato specificamente sul dataset di progetto
- ✅ Valutazione oggettiva tramite metriche standard (Accuracy, F1, Precision, Recall)
- ✅ Inferenza funzionante e confrontabile per entrambi i modelli
- ✅ Visualizzazione dei risultati tramite confusion matrix

Dal confronto emerge come, in questo scenario specifico, **FastText ottenga prestazioni migliori in termini di Macro-F1**, evidenziando come modelli più leggeri ma addestrati su dati coerenti possano risultare competitivi rispetto a modelli più complessi.

### Prossimi passi
- Deploy dell’applicazione su **Hugging Face Spaces**
- Integrazione di un sistema di **monitoring continuo** con Evidently AI
- Automazione del **retraining del modello** all’arrivo di nuovi dati
